## Investigate Source Datasets

In [1]:
# import main libraries
import datetime
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
from collections import Counter
plt.style.use('fivethirtyeight')
%matplotlib inline

In [2]:
# load and read our dataset for initial exploration - from data.world (source for Tableau dashboard)
df = pd.read_csv('https://query.data.world/s/cg3texvvmoyagywal62azempdhhyu4')

# preview our data
df.head(5)

/Users/shim/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Table_Names,Date,Case_Type,Cases,Difference,Country_Region,Province_State,Admin2,Combined_Key,FIPS,Lat,Long,Latest_Date,Prep_Flow_Runtime
0,Daily Summary,3/27/2020,Deaths,0.0,NaN,US,Virgin Islands,NaN,"Virgin Islands, US",78000.0,18.3358,-64.8963,3/27/2020,3/27/2020 10:27:00 PM
1,Daily Summary,3/27/2020,Confirmed,19.0,NaN,US,Virgin Islands,NaN,"Virgin Islands, US",78000.0,18.3358,-64.8963,3/27/2020,3/27/2020 10:27:00 PM
2,Daily Summary,3/27/2020,Deaths,3.0,NaN,US,Puerto Rico,NaN,"Puerto Rico, US",NaN,18.2208,-66.5901,3/27/2020,3/27/2020 10:27:00 PM
3,Daily Summary,3/27/2020,Confirmed,79.0,NaN,US,Puerto Rico,NaN,"Puerto Rico, US",NaN,18.2208,-66.5901,3/27/2020,3/27/2020 10:27:00 PM
4,Daily Summary,3/27/2020,Deaths,1.0,NaN,US,Guam,NaN,"Guam, US",66000.0,13.4443,144.7937,3/27/2020,3/27/2020 10:27:00 PM


In [9]:
# compare to alternate dataset - from ecdc
date = datetime.datetime.now()
COVID_19_URL = "https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-"
df2=pd.read_excel(COVID_19_URL + date.strftime("%Y-%m-%d")+ ".xlsx", sep = "")

# preview  data
df2.head(5)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
0,2020-03-28,28,3,2020,16,1,Afghanistan,AF,AFG,37172386.0
1,2020-03-27,27,3,2020,0,0,Afghanistan,AF,AFG,37172386.0
2,2020-03-26,26,3,2020,33,0,Afghanistan,AF,AFG,37172386.0
3,2020-03-25,25,3,2020,2,0,Afghanistan,AF,AFG,37172386.0
4,2020-03-24,24,3,2020,6,1,Afghanistan,AF,AFG,37172386.0


In [38]:
total_by_country2 = df2[['dateRep', 'countriesAndTerritories','deaths']]
total_by_country2

,dateRep,countriesAndTerritories,deaths
0,2020-03-28,Afghanistan,1
1,2020-03-27,Afghanistan,0
2,2020-03-26,Afghanistan,0
3,2020-03-25,Afghanistan,0
4,2020-03-24,Afghanistan,1
...,...,...,...
7315,2020-03-25,Zimbabwe,0
7316,2020-03-24,Zimbabwe,1
7317,2020-03-23,Zimbabwe,0
7318,2020-03-22,Zimbabwe,0


In [50]:
total_deaths2 = total_by_country2['deaths'].sum()
total_deaths2

26990

In [51]:
# compare to direct link to John Hopkins timeset data
data = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
df3 = pd.read_csv(data, sep=",")

# preview our data
df3.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,0,0,0,0,1,1,1,2,4,4
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2,2,2,2,2,4,5,5,6,8
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7,9,11,15,17,17,19,21,25,26
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,3,3
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
total_deaths3 = df3.iloc[:, 4:].sum()
total_deaths3

1/22/20       17
1/23/20       18
1/24/20       26
1/25/20       42
1/26/20       56
           ...  
3/23/20    16505
3/24/20    18625
3/25/20    21181
3/26/20    23970
3/27/20    27198
Length: 66, dtype: int64

In [53]:
# what's the difference between the 2 databases? 
diff = total_deaths3.max() - total_deaths2
diff

208

## Investigate selected Dataset

#### Using John Hopkins data - because it's the most deaths 

In [60]:
# making copy and renaming master file
covid = df3.copy()
covid.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,0,0,0,0,1,1,1,2,4,4
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2,2,2,2,2,4,5,5,6,8
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,7,9,11,15,17,17,19,21,25,26
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,3,3
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
# set date searched
from datetime import datetime, timedelta

today = datetime.now()
yesterday = today - timedelta(1)

# INPUT
date = yesterday
date = date.strftime('%m/%d/%y').lstrip("0").replace(" 0", " ")
date

'3/27/20'

In [64]:
# sanity check
covid.shape

(249, 70)

In [65]:
covid.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20
count,249.00000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,...,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000,249.000000
mean,22.27243,24.439131,0.068273,0.072289,0.104418,0.168675,0.224900,0.329317,0.526104,0.534137,...,35.072289,39.626506,45.377510,52.100402,58.839357,66.285141,74.799197,85.064257,96.265060,109.228916
std,24.55691,71.376441,1.077331,1.078938,1.523056,2.535964,3.296752,4.817222,7.921510,7.921989,...,284.278265,307.679074,342.127762,387.752366,429.889003,473.801267,526.636872,586.324407,648.648777,724.792679
min,-41.45450,-135.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.53800,-15.180400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,24.00000,21.005900,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
75%,41.37750,88.092400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,2.000000,3.000000,3.000000,4.000000,5.000000,6.000000,6.000000
max,71.70690,178.065000,17.000000,17.000000,24.000000,40.000000,52.000000,76.000000,125.000000,125.000000,...,3122.000000,3405.000000,4032.000000,4825.000000,5476.000000,6077.000000,6820.000000,7503.000000,8215.000000,9134.000000


In [77]:
# total deaths
total_deaths = covid[[date]].sum()
total_deaths

3/27/20    27198
dtype: int64